In [1]:
# !pip install torchvision~=0.15.1 --force-reinstall --extra-index-url https://download.pytorch.org/whl/cu118
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip install autogluon

In [2]:
import torch
print(torch.cuda.is_available())  # Should be True
print(torch.cuda.device_count())  # Should be > 0

True
1


## Import

In [3]:
import os
import time
from pprint import pprint

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import random

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

from sklearn.preprocessing import LabelEncoder

In [4]:
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data.dataset import random_split

In [5]:
from util import *
DATA_ADDRESS 

'./data/'

In [6]:
torch.cuda.is_available()

True

In [7]:
from datetime import datetime

print("Begining Time:\n", datetime.now())

Begining Time:
 2024-02-12 20:58:11.170976


## Settings

In [8]:
# Random seeds
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = "cpu"


# Hyperparameters
random_seed = 42
learning_rate = 10e5
num_epochs = 10
batch_size = 64

# Architecture
num_classes = 200



## Dataset

In [9]:
# X,y= load_data(
#     dir_feature = TRAIN_DIR,
#     file_prefix = 'source_mfcc_len5_fft2048_mels128_mfcc17_',
#     dir_df_index = os.path.join(DATA_ADDRESS,'df_index_source_train.pkl'),
#     n_interval=500,
#     flatten=False)

In [10]:
data= load_data(
    dir_feature = TRAIN_DIR,
    file_prefix = 'source_mfcc_len5_fft2048_mels128_mfcc17_',
    dir_df_index = os.path.join(DATA_ADDRESS,'df_index_source_train.pkl'),
    n_interval=500,
    flatten=True)

100%|██████████| 4/4 [00:00<00:00, 499.96it/s]


In [11]:
from sklearn.model_selection import train_test_split
data_train,data_valid = train_test_split(
    data,
    # data.drop('target', axis=1),  # Assuming 'target' is the column with the labels
    # data['target'],
    test_size=0.2,  # 20% for validation
    random_state=42  # For reproducibility
)


## Model

In [16]:
from autogluon.tabular import TabularPredictor
from tqdm.notebook import tqdm
from tqdm.autonotebook import tqdm as notebook_tqdm

tqdm.pandas()
predictor = TabularPredictor(label='target').fit(data_train,presets='high_quality')

No path specified. Models will be saved in: "AutogluonModels\ag-20240213_020116"
Presets specified: ['high_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 3600 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels\ag-20240213_020116/ds_sub_fit/sub_fit_ho.
2024-02-12 21:01:16,754	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
Beginning AutoGluon training ... Time limit 

In [17]:
predictor

In [18]:
predictor.evaluate(data_valid, silent=True)

{'root_mean_squared_error': -2593.272216796875,
 'mean_squared_error': -6725060.5,
 'mean_absolute_error': -2266.455322265625,
 'r2': 0.10801174642735833,
 'pearsonr': 0.3313117953906841,
 'median_absolute_error': -2236.072998046875}